In [5]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import json
import numpy as np
from numpy import zeros
from numpy import asarray
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence,one_hot
import codecs

embed_index = {}
glove_vector_path = 'drive/MC/glove.6B.100d.txt'
def get_glove_values():
  with codecs.open(glove_vector_path,'rb',encoding='utf-8') as f:
      print("========> Importing GloVe Vector  <=========")
      #Parsing GloVe dataset
      for line in f:
          values = line.split()
          word = values[0].lower()
          embed_index[word] = np.asarray(values[1:], dtype='float32')
      print("Number of vectors loaded : "+ str(len(embed_index)))

In [8]:
get_glove_values()

========> Importing GloVe Vector  <=========
Number of vectors loaded : 400000


In [0]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, merge
from keras.models import Sequential
import tensorflow as tensor

CONST_INPUT_SIZE = 1

vocab_context={}
vocab_question={}

def tokenize_and_embed_words(inputValue,isContext):
  words_in_input = text_to_word_sequence(inputValue)
  count_words_input = 0
  if isContext :
    vocab_context = np.zeros((len(words_in_input), 100))
    for word in words_in_input:
      if word in embed_index.keys():
        vocab_context[count_words_input] = embed_index[word]
        count_words_input += 1
    actual_data = vocab_context[0:count_words_input]
  else:
    vocab_question = np.zeros((len(words_in_input), 100))
    for word in words_in_input:
      if word in embed_index.keys():
        vocab_question[count_words_input] = embed_index[word]
        count_words_input += 1
    actual_data = vocab_question[0:count_words_input]
  return actual_data 
    
def import_squad_dataset():
    #To parse the glove values in to an array form
    print("Step 1: Started Context Tokenization...")
    squad_data_set='drive/MC/Dev.json'
    with codecs.open(squad_data_set,'rb',encoding='utf-8') as data_file:
        data = json.load(data_file)
        data_entity_list = []
        count_iterations = 0
        question='';
        for item in data['data']:
          paragraph = item['paragraphs']
          for ctx in paragraph:
            if count_iterations < CONST_INPUT_SIZE:
               context = ctx['context']
               vocab_context = tokenize_and_embed_words(context,True);
               print(vocab_context)
               qas = ctx['qas']
               for question_text in qas:
                 question = question_text['question']
                 vocab_question = tokenize_and_embed_words(question,False);
                 break
               # model1
               size_context = len(vocab_context)
               size_question = len(vocab_question)
               
               vocab_context_tensor = tensor.convert_to_tensor(vocab_context,dtype=tensor.float32)
               embed_context = Embedding(input_dim=size_context, output_dim=100,input_length=size_context,trainable=False)
               tensor_context = embed_context(vocab_context_tensor)
               lstm_context = Bidirectional(LSTM(100))
               lstm_context_out = lstm_context(tensor_context)
               print(lstm_context_out)
               #drop_1 = Dropout(0.5)(lstm_out)

               # model2
               vocab_question_tensor = tensor.convert_to_tensor(vocab_question,dtype=tensor.float32)
               embed_question = Embedding(input_dim=size_question, output_dim=100,input_length=size_question,trainable=False)
               tensor_context = embed_context(vocab_question_tensor)
               lstm_question = Bidirectional(LSTM(100))
               lstm_question_out = lstm_question(tensor_context)
               print(lstm_question_out)
                
               #merged_tensor = merge([lstm_context_out, lstm_question_out], mode='dot',dot_axes=-1)
               #print(merged_tensor.summary())
               
               #model = Sequential()
               #model.add(Merge([lstm_context_out, lstm_question_out], mode='concat'))
               #model.add(Dense(1))
               #model.add(Activation('sigmoid'))
               #model.compile(optimizer='RMSprop', loss='binary_crossentropy')
               #model.fit([X[:,:m,:], X[:,m:,:]], y)
               #ques_input = Input(shape=(50, ), dtype='int32', name='ques_input')
               #x = Embedding(input_dim=vocab_size, output_dim=50, weights=[vocab_question],input_length=50)
               #lstm_out = Bidirectional(LSTM(256, return_sequences=True, implementation=2), merge_mode='concat')(x)
               #drop_2 = Dropout(0.5)(lstm_out) 
              
              
              #words_in_paragraph = tokenize_text(context)
               #words_in_paragraph = text_to_word_sequence(context)
               #vocab_context = np.zeros((len(words_in_paragraph), 100))
               #vocab_question = np.zeros((len(words_in_paragraph), 100
               #count_words_question = 0
               #for word in words_in_paragraph:
                  #if word in embed_index.keys():
                 #   vocab_context[count_words_question] = embed_index[word]
                  #  count_words_question += 1
               #actual_data = vocab_context[0:count_words_question]
               #model = Sequential()
               #model.add(Embedding(input_dim=count_words_question,output_dim=count_words_question, input_length=100))
               #e = Embedding(count_words_question, 100, weights=actual_data)
               #model.add(e)
               #model=(Bidirectional(LSTM(count_words_question,use_bias=True)))
               #model.add(Dense(count_words_question, activation='sigmoid'))
               # compile the model
               #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
               # summarize the model
               #print(model)
               #print(actual_data)
               count_iterations += 1

In [28]:
import_squad_dataset()

Step 1: Started Context Tokenization...
[[-0.44532999 -0.44969001  1.23930001 ...  0.65855998  0.79881001
  -0.16019   ]
 [-0.80625999  0.36656001  0.46963999 ...  0.91166002  1.12919998
   0.71038997]
 [ 0.070568    0.55312002 -0.055536   ...  0.18748     0.64086002
  -0.15872   ]
 ...
 [ 0.26333001  0.41064    -0.35883999 ... -1.01520002  0.33160001
   0.83611   ]
 [-0.26717001  0.74791002 -0.11579    ... -0.54696    -0.62404001
   0.3809    ]
 [ 0.070568    0.55312002 -0.055536   ...  0.18748     0.64086002
  -0.15872   ]]
Tensor("bidirectional_17/concat:0", shape=(123, 200), dtype=float32)
Tensor("bidirectional_18/concat:0", shape=(10, 200), dtype=float32)


In [46]:
print(vocab_context)
print("Question Vector")
print(vocab_question)

{}
Question Vector
{}
